# Gemma-3 270M LoRA / QLoRA Quickstart (Colab)

This notebook helps you set up a Colab GPU runtime, install required packages, provide your Hugging Face token, choose a model variant, and run a small dry-run of the project's training CLI in `--mode lora` or `--mode qlora`.

Open the runtime type as `GPU` before running the install cell.

## 1) Install dependencies (run in a GPU runtime)
Run the cell below to install the core dependencies. For QLoRA you also need a compatible CUDA + bitsandbytes build.

In [ ]:
# Install core dependencies. This may take a few minutes in Colab.
!pip install --upgrade pip
!pip install transformers accelerate datasets safetensors
!pip install -r requirements-dev.txt
# bitsandbytes and peft are optional but useful for LoRA/QLoRA runs; the pip may fail on some envs, so allow failure
!pip install git+https://github.com/huggingface/peft.git || true
!pip install bitsandbytes || true
print('Install step finished (errors for bitsandbytes may be expected on some Colab CUDA versions).')

## 2) Provide your Hugging Face token
Run the cell and paste your HF token when prompted. The token is stored in the environment for the notebook session only.

In [ ]:
from getpass import getpass
import os
token = getpass('Enter your Hugging Face token (input hidden): ')
if token:
  os.environ['HF_TOKEN'] = token
  print('HF token set in environment for this session')
else:
  print('No token provided; some actions may be skipped')

## 3) Choose model variant and mode
Use the next cell to pick either `google/gemma-3-270m` (default) or another compatible Gemma checkpoint. Toggle QLoRA if you want to attempt 4-bit quantized flow (only if bitsandbytes is installed).

In [ ]:
# Default model id (change if you have a different HF checkpoint)
model_id = 'google/gemma-3-270m'
# Set to True to attempt QLoRA (requires bitsandbytes + CUDA)
use_qlora = False
# Example overrides: model_id = 'google/gemma-3-270m'
print(f'Model: {model_id}, QLoRA enabled: {use_qlora}')

## 4) Quick check: CUDA / wheel helper
This cell inspects the runtime CUDA version (if available) and prints recommended pip commands for installing PyTorch and bitsandbytes suitable for common Colab CUDA versions. Use the printed command to (re)install if needed.

In [ ]:
import subprocess, sys, re
def get_nvidia_smi():
    try:
        out = subprocess.check_output(['nvidia-smi','--query-gpu=driver_version,cuda_version','--format=csv,noheader'])
        return out.decode().strip()
    except Exception as e:
        return None
info = get_nvidia_smi()
print('nvidia-smi info:', info)
cuda_tag = 'cpu'
if info:
    m = re.search(r'([0-9]+[0-9]+)', info)
    if m:
        v = m.group(1)
        # Map common Colab CUDA to torch wheel tags (best-effort)
        if v.startswith('12.1') or v.startswith('12.0'):
            cuda_tag = 'cu121'
        elif v.startswith('11.8'):
            cuda_tag = 'cu118'
        elif v.startswith('11.7'):
            cuda_tag = 'cu117'
        else:
            cuda_tag = 'cu118'
print('Detected CUDA tag suggestion:', cuda_tag)
if cuda_tag != 'cpu':
    print('Recommended pip install (example):')
    print(f'pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/{cuda_tag}')
    print(f'pip install bitsandbytes')
else:
    print('CPU-only mode detected (no CUDA), LoRA may still work but will be slower')

## 5) Run a LoRA/QLoRA training smoke test
This cell runs a minimal training test using the project's CLI. Adjust `--max_train_steps` and `--dataset_name` as needed for a quick validation.

In [ ]:
# Run a small LoRA training test
import subprocess
import os

# Set mode based on the use_qlora flag
mode = 'qlora' if use_qlora else 'lora'
print(f'Running training in {mode} mode...')

# Basic training command - adjust parameters as needed
cmd = [
    'python', 'scripts/finetune_gemma_from_csv.py',
    '--csv', 'synthetic_wifi_5ghz_outdoor_smoke.csv',
    '--mode', mode,
    '--base-model', model_id,
    '--mode', mode,
    '--num-epochs', '1',  # Very short for smoke test
    '--max-rows', '100',  # Limit rows for quick test
    '--checkpoint-dir', 'outputs/colab-smoke-test'
]

# Run the training
try:
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
    print('STDOUT:')
    print(result.stdout)
    if result.stderr:
        print('STDERR:')
        print(result.stderr)
    print(f'Training finished with return code: {result.returncode}')
except subprocess.TimeoutExpired:
    print('Training timed out after 5 minutes - this is expected for a smoke test')
except Exception as e:
    print(f'Error running training: {e}')

## 6) Check training results
This cell lists the output directory to see what files were created during training.

In [ ]:
import os
from pathlib import Path

output_dir = Path('outputs/colab-smoke-test')
if output_dir.exists():
    print(f'Contents of {output_dir}:')
    for item in output_dir.rglob('*'):
        if item.is_file():
            print(f'  {item.relative_to(output_dir)} ({item.stat().st_size} bytes)')
else:
    print(f'Output directory {output_dir} does not exist')

## 7) Push trained model to Hugging Face Hub (Optional)
This cell will push a local model directory to the Hub. Ensure `HF_TOKEN` is set in the session and you have `huggingface_hub` installed. Edit `HUB_REPO_ID` to a repo you control (e.g., 'your-username/gemma3-finetuned').

In [ ]:
# Push model to HF Hub (requires HF_TOKEN and huggingface_hub installed)
import os
from pathlib import Path

HUB_REPO_ID = 'your-username/gemma3-finetuned'  # EDIT_THIS
MODEL_DIR = 'outputs/colab-smoke-test'  # set to your final model dir

if not os.environ.get('HF_TOKEN'):
    print('HF_TOKEN not set in environment; set it and re-run this cell')
elif not Path(MODEL_DIR).exists():
    print(f'Model directory {MODEL_DIR} does not exist; train a model first')
else:
    try:
        # Install huggingface_hub if needed
        try:
            from huggingface_hub import HfApi
        except ImportError:
            import subprocess
            subprocess.check_call(['pip', 'install', 'huggingface_hub'])
            from huggingface_hub import HfApi
        
        api = HfApi()
        print('Creating or ensuring repo exists...')
        api.create_repo(HUB_REPO_ID, private=False, exist_ok=True, token=os.environ['HF_TOKEN'])
        
        print('Uploading model files...')
        api.upload_folder(
            folder_path=MODEL_DIR,
            repo_id=HUB_REPO_ID,
            token=os.environ['HF_TOKEN'],
            commit_message='Add fine-tuned Gemma adapter'
        )
        print(f'Successfully pushed model to {HUB_REPO_ID}')
    except Exception as e:
        print(f'Failed to push to Hub: {e}')